# Домашняя работа №1 Нужненко Александры, БКЛ-202

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
import pymystem3
mystem=pymystem3.Mystem()

In [3]:
import nltk
from nltk.tokenize import word_tokenize

In [80]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [197]:
from nltk.corpus import stopwords

# загружаем список стоп-слов на русском
sw = stopwords.words('russian')

In [198]:
from sklearn.metrics import accuracy_score

In [4]:
session = requests.session() # открытие сессии
# ф-я принимает ссылку и возвращает html код страницы
def get_page(url):
        response = session.get(url)
        return response.text

In [6]:
# ф-я принимает html и возвращает меньшие блоки кода, содержащие положительный отзыв
def get_block_good(html):
    soup = BeautifulSoup(html, 'html.parser')
    blocks = soup.find_all("div", {'class': 'response good'})
    return blocks

In [7]:
# ф-я принимает html и возвращает меньшие блоки кода, содержащие отрицательный отзыв
def get_block_bad(html):
    soup = BeautifulSoup(html, 'html.parser')
    blocks = soup.find_all("div", {'class': 'response bad'})
    return blocks

In [33]:
# ф-я принимает список блоков html кода и возвращает список текстов отзывов
def get_text(html_block):
    texts_li = []
    for block in html_block:
        # здесь .split() и " ".join нужны для того, чтобы неразрывные пробелы заменить обычными
        texts_li.append(" ".join(block.find_all("span", {'class': '_reachbanner_'})[0].get_text().split()))
    return texts_li

In [93]:
# ф-я принимает строку текста и возвращает список слов в нижнем регистре, в начальной форме, без пунктуации
def clear_data(text):
    tokens = word_tokenize(text) # токенизация
    clean_data = []
    for token in tokens:
        ana = morph.parse(token)
        if ana[0].tag.POS != None: # избавляемся от пунктуации
            clean_data.append(ana[0].normal_form) # добавляем леммы в результирующий список
    return clean_data

In [109]:
# ф-я принимает список всех слов и возвращает список слов, в котором нет стоп-слов 
def delete_stops(li_words):
    without_stops = []
    for word in li_words:
        if word not in sw:
            without_stops.append(word)
    return without_stops

In [138]:
# ф-я принимает список слов и возвращает отсортированный по значению словарь вида {слово : кол-во раз в списке}
def sorted_dct_words_num(li_words):
    dct = {}
    dct_sorted = {}
    for word in li_words:
        if word not in dct:
            dct[word] = 1
        else:
            dct[word] += 1
            
    sorted_keys = sorted(dct, key=dct.get)
    for element in sorted_keys:
        dct_sorted[element] = dct[element]
        
    return dct_sorted

In [163]:
# ф-я принимает множество слов, словарь вида {слово : кол-во раз, которое встретилось это слово} 
# и число, которое отражает частоту слова в отзыве; 
# слова с этой и меньшей частотой удаляются, потому что считаются неинформативными
# ф-я возвращает множество слов, которые (как предполагается) являются информативными
def delete_uninformative_words(set_words, dct, number):
    set_informative = set()
    for word in list(set_words):
        if dct[word] > number:
            set_informative.add(word)
    return set_informative     

In [179]:
# ф-я принимает текст комментария; если комментарий положительный, возвращает True, если отрицательный - False 
def type_of_comment(text):
    positive_count = 0
    negative_count = 0
    li_words = clear_data(text)
    for word in li_words:
        if word in set_informative_good:
            positive_count += 1
        if word in set_informative_bad:
            negative_count += 1
    if positive_count > negative_count:
        return True
    else:
        return False

In [225]:
def results(li_test, li_standard):
    results = []
    for text in test:
        if type_of_comment(text):
            results.append("pos")
        else:
            results.append("neg")
    print("RESULTS:")
    print("Accuracy: %.4f" % accuracy_score(results, li_standard))

Возьмем 4 страницы с рецензиями на фильмы в качестве данных. С первой страницы возьмем как положительные, так и отрицательные отзывы, со страниц 2-4 возьмем только отрицательные отзывы, чтобы количество позитивных и негативных рецензий было примерно равным.

In [22]:
page1 = get_page("https://www.kinopoisk.ru/reviews/type/comment/period/month/perpage/200/#list")

In [20]:
page2 = get_page("https://www.kinopoisk.ru/reviews/type/comment/period/month/page/2/#list")

In [36]:
page3 = get_page("https://www.kinopoisk.ru/reviews/type/comment/period/month/page/3/#list")

In [49]:
page4 = get_page("https://www.kinopoisk.ru/reviews/type/comment/period/month/page/4/#list")

Выделяем блоки html-кода, в котором содержатся тексты рецензий.

In [40]:
blocks_good = get_block_good(page1)

In [50]:
blocks_bad = get_block_bad(page1) + get_block_bad(page2) + get_block_bad(page3) + get_block_bad(page4)

Выделяем тексты комментариев.

In [42]:
texts_good = get_text(blocks_good) # все ПОЛОЖИТЕЛЬНЫЕ отзывы

In [51]:
texts_bad = get_text(blocks_bad) # все ОТРИЦАТЕЛЬНЫЕ отзывы

In [232]:
print("Всего собрали", len(texts_good), "положительных рецензий.")

Всего собрали 128 положительных рецензий.


In [233]:
print("Всего собрали", len(texts_bad), "отрицательных рецензий.")

Всего собрали 137 отрицательных рецензий.


25% из каждой группы оставим для проверки качества:

In [220]:
test_good = texts_good[:int(len(texts_good)/100*25)]

In [221]:
test_bad = texts_bad[:int(len(texts_bad)/100*25)]

In [222]:
standard = len(test_good) * ["pos"] + len(test_bad) * ["neg"] # стандарт, к которому стремимся 

In [227]:
test = test_good + test_bad # данные для проверки вместе

Остальные комментарии - материал для выделения положительно и отрицательно окрашенных слов:

In [70]:
learn_good = texts_good[int(len(texts_good)/100*25):]

In [72]:
learn_bad = texts_bad[int(len(texts_bad)/100*25):]

Делаем из текстов списки слов в начальной форме, в нижнем регистре и без знаков препинания:

In [101]:
li_words_good = []
for text in learn_good:
    li_words_good += clear_data(text)

In [103]:
li_words_bad = []
for text in learn_bad:
    li_words_bad += clear_data(text)

Удаляем стоп-слова:

In [112]:
li_words_good = delete_stops(li_words_good)

In [113]:
li_words_bad = delete_stops(li_words_bad)

Создаем словари с подсчетом частотности слов:

In [140]:
dct_good_sorted = sorted_dct_words_num(li_words_good)

In [141]:
dct_bad_sorted = sorted_dct_words_num(li_words_bad)

Создаем множества слов, которые есть в положительных и отрицательных рецензиях:

In [114]:
set_words_good = set(li_words_good)

In [115]:
set_words_bad = set(li_words_bad)

Создаем множества слов, которые есть ТОЛЬКО в положительных и ТОЛЬКО отрицательных рецензиях:

In [119]:
set_only_good = set_words_good - set_words_bad

In [121]:
set_only_bad = set_words_bad - set_words_good

Посмотрим объем получившихся множеств:

In [124]:
len(set_only_good)

3420

In [125]:
len(set_only_bad)

3765

Уберем "шум":

In [246]:
set_informative_good = delete_uninformative_words(set_only_good, dct_good_sorted, 5)

In [247]:
set_informative_bad = delete_uninformative_words(set_only_bad, dct_bad_sorted, 5)

Вычислим accuracy:

In [248]:
results(test, standard)

RESULTS:
Accuracy: 0.5303


При исключении слов 
с частотой 1 accuracy = 0.6212,
с частотой 2 accuracy = 0.6515, 
с частотой 3 accuracy = 0.6667, 
с частотой 4 accuracy = 0.5606,
с частотой 5 accuracy = 0.5303

### Итоги: 
Лучший показатель accuracy достигается, когда исключаем слова с частотностью 3. В целом показатель не очень хороший. Скорее всего это связано с тем, что в множестве set_informative_good (предполагаемо информативные слова) почти нет слов, которые действительно положительно окрашены, а в множестве set_informative_bad очень мало действительно отрицательно окрашенных слов. Требуются дополнительные модификации метода для улучшения результата.

### Как улучшить программу:

1. Убрать из стоп-слов частицу "не", и при работе со списком слов, встречающихся в комментариях, склеивать "не" и следующее слово (например, "единой формой" станут "не понравиться", "не зацепить" и т.д.) Это позволит лучше отслеживать отношение пишущего, поскольку частица "не" меняет смысл на противоположный, и при ее исключении получается, что комментарии, где изначально было написано "мне понравилось" и где было написано "мне НЕ понравилось", содержат одинаковые элементы "понравиться", которые затем исключаются из множества, т.к. считается, что они не несут окраску, поскольку встретились и в положительном, и в отрицательном тексте.

2. Увеличить объем материала, на основе которого пытаемся проводить анализ. Это увеличит объем словаря и добавит новые варианты эмоционально окрашенных слов.

3. Ввести более сложное условие для определения того, положительный отзыв нам попался или отрицательный. Например, можно ввести формулу с разными весами для разных частей речи (e.g., если в тексте отзыва встретилось прилагательное из сета положительно окрашенных слов, то в счетчик добавляется три балла, если встретилось существительное, - то два балла, если глагол - лдин балл).